<a href="https://colab.research.google.com/github/zia207/python-colab/blob/main/NoteBook/Machine_Learning/Tree_based/03-01-02-08-03-tree-based-models-bagging-grf-causal-survival-forest-python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://drive.google.com/uc?export=view&id=1IFEWet-Aw4DhkkVe1xv_2YYqlvRe9m5_)


# 2.8.3 Causal Survival Forest (CSF)

This notebook demonstrates how to use the **Causal Survival Forest (CSF)** from the `{grf}` package in R to estimate heterogeneous treatment effects on survival data, particularly when outcomes are right-censored. The example uses the `lung` dataset, which contains survival data for patients with advanced lung cancer.

## Overview

A **Causal Survival Forest (CSF)** is designed to estimate **heterogeneous treatment effects** for survival data, particularly when outcomes are right-censored. It extends the concept of causal forests to handle time-to-event data, enabling researchers to estimate how a treatment impacts survival outcomes across different subgroups defined by covariates, without relying on restrictive parametric assumptions. CSFs are useful in fields like medicine, economics, and social sciences, where understanding how treatments affect survival outcomes across diverse populations is critical (e.g., estimating the effect of a drug on patient survival based on age, sex, or disease severity).



### Key Concepts of Causal Survival Forests

1.  `Heterogeneous Treatment Effects`: CSFs estimate the **conditional average treatment effect (CATE)**, denoted as $\tau(X)$, which varies across individuals based on their covariate profile $X$. For survival data, this effect can be defined in two ways:

-   `Restricted Mean Survival Time (RMST)`: $\tau(X) = E[\min(T(1), h) - \min(T(0), h) | X = x]$, where $T(1)$ and $T(0)$ are the potential survival times under treatment and control, respectively, and $h$ is a specified time horizon.

-   `Survival Probability`: $\tau(X) = P[T(1) \> h \| X = x] - P[T(0) \> h \| X = x$, which compares the probability of surviving past time $h$ under treatment versus control.

2.  `Right-Censoring`: Survival data often involves censoring, where the event time is not observed for all subjects. CSFs account for this by incorporating ensoring indicators and using techniques like inverse probability of censoring weighting (IPCW).

3.  `Non-parametric Approach`: Unlike traditional survival models (e.g., Cox proportional hazards), CSFs are non-parametric, making them flexible for capturing complex, non-linear relationships between covariates and treatment effects.

4.  `Forest-Based Estimation`: CSFs build an ensemble of trees, where each tree splits the covariate space to maximize heterogeneity in treatment effects. Predictions are made by aggregating estimates across trees, weighted by how often a test sample falls into the same leaf as training samples.

5.  `Honest Estimation`: The `grf` package uses "honest" estimation, splitting the data into subsets for choosing tree splits and populating leaf estimates, reducing overfitting.

6.  `Applications`: CSFs are useful in fields like medicine, economics, and social sciences, where understanding how treatments affect survival outcomes across diverse populations is critical (e.g., estimating the effect of a drug on patient survival based on age, sex, or disease severity).

### How CSFs Work

1.  Input Data

-   $X$: Covariates (e.g., age, sex, clinical measurements).

-   $Y$: Observed time (minimum of event time and censoring time).

-   $W$: Treatment assignment (binary or continuous).

-   $D$: Censoring indicator (1 if the event is observed, 0 if censored).

-   $h$: Time horizon for RMST or survival probability.

2.  Splitting Criterion

-   Trees are grown by maximizing the difference in treatment effects between child nodes, using a modified objective that accounts for censoring (e.g., via AIPW scores, as described in Cui et al., 2023).

3.  Prediction

-   For a given covariate profile (X = x), the forest estimates $\tau(x)$ by averaging treatment effect estimates from trees, using weights based on sample: CSFs use techniques like IPCW or survival forests to estimate censoring probabilities, ensuring unbiased treatment effect estimates.


### How does it differ from Causal Forest and Survival Forest?



| **Aspect**                | **Survival Forest**                              | **Causal Forest**                              | **Causal Survival Forest**                     |
|---------------------------|--------------------------------------------------|-----------------------------------------------|-----------------------------------------------|
| `Objective`             | Predict survival probability or hazard over time | Estimate heterogeneous treatment effects       | Estimate treatment effects on survival outcomes |
| `Outcome Type`          | Time-to-event (censored)                        | Continuous or binary                          | Time-to-event (censored)                      |
| `Treatment Variable`    | None                                            | Required (W: 0/1)                            | Required (W: 0/1)                             |
| `Inputs`                | X, Y, D                                         | X, Y, W                                      | X, Y, D, W                                    |
| `Output`                | Survival curves or hazard functions             | Treatment effect estimates $\tau(X)$)      | Treatment effect on survival $\tau(t | X)$ |
| `Use Case`              | Survival prediction (e.g., patient prognosis)   | Treatment effect heterogeneity (e.g., drug impact) | Treatment effect on survival (e.g., therapy impact) |
| `Assumptions`           | Handles censoring                               | Unconfoundedness, overlap                    | Unconfoundedness, overlap, censoring handled  |
| `{grf} Function`        | `survival_forest()`                             | `causal_forest()`                            | `causal_survival_forest()`                    |


### Practical Notes

- `Survival Forest`: Used when the goal is purely predictive (e.g., forecasting patient survival based on covariates). It does not involve a treatment variable.

- `Causal Forest`: Used for non-survival outcomes (e.g., blood pressure, test scores) to understand how treatment effects vary across individuals.

`Causal Survival Forest`: Used when both treatment effects and survival outcomes are of interest, combining the strengths of the other two models.


## Causal Survival Forest with Python

A custom Python implementation of Causal Survival Forests (CSF) is essential because Python lacks robust native alternatives to R’s `grf` for survival-based causal inference. While libraries like `econml` offer some functionality, they often lack stability or full feature parity. CSF combines survival analysis (handling censored data) with causal ML to estimate individualized treatment effects—critical for healthcare, marketing, and engineering applications. A custom solution ensures Python compatibility, flexibility for research, and scalability while avoiding R dependencies. This bridges a key gap in Python’s causal ML ecosystem, enabling production-ready survival analysis with heterogeneous treatment effects.



This code implements a `CausalSurvivalForest` class, a Python adaptation of the generalized random forest (GRF) causal survival forest from R, designed to estimate causal treatment effects in survival analysis. Here's a brief explanation:

### Purpose
The `CausalSurvivalForest` estimates heterogeneous treatment effects on survival outcomes, accounting for censoring, using a combination of random forests and survival forests. It predicts how a treatment impacts survival probabilities across different covariate profiles.

### Key Components
1. **Imports and Dependencies**:
   - Uses `numpy` for numerical operations, `sklearn` for random forest models, and `sksurv` for survival analysis.
   - Imports `RandomForestRegressor` for propensity scores and treatment effects, and `RandomSurvivalForest` for survival modeling.

2. **Class Structure**:
   - Inherits from `sklearn.base.BaseEstimator` for compatibility with scikit-learn's API.
   - Parameters include:
     - `n_estimators`: Number of trees in the forest (default: 1000).
     - `min_samples_leaf`: Minimum samples per leaf (default: 10).
     - `max_depth`: Maximum tree depth (default: None, unlimited).
     - `random_state`: For reproducibility.
     - `honesty` and `alpha`: Control honest splitting for unbiased estimates.
     - `verbose`: Toggles progress logging.

3. **Models**:
   - `model_t`: A `RandomForestRegressor` to estimate propensity scores (probability of receiving treatment).
   - `model_y`: A `RandomSurvivalForest` to model survival probabilities, incorporating treatment as a feature.
   - `model_tau`: A `RandomForestRegressor` to estimate treatment effects (difference in survival outcomes).

4. **Key Methods**:
   - **`fit(X, W, Y, D, sample_weight=None)`**:
     - Inputs: Features (`X`), binary treatment (`W`), observed times (`Y`), event indicators (`D`), and optional sample weights.
     - Steps:
       1. Fits `model_t` to estimate propensity scores (`P(W=1|X)`), clipped to `[0.1, 0.9]` to avoid extreme values.
       2. Fits `model_y` to predict survival probabilities, using `X` and `W` as features.
       3. Computes survival curves for treated (`W=1`) and control (`W=0`) groups, calculates the restricted mean survival time (RMST) difference as the treatment effect (`tau`), and fits `model_tau` to predict `tau` across `X`.
     - Stores unique event times for later predictions.

   - **`_predict_survival(X, W)`**:
     - Predicts survival probabilities for a given treatment `W` (0 or 1) at the unique event times from training.

   - **`predict(X)`**:
     - Predicts treatment effects (`tau`) for new data `X` using `model_tau`.

   - **`predict_survival(X, W, times=None)`**:
     - Predicts survival probabilities for new data `X` under treatment `W` at specified times (or training event times if unspecified).

   - **`get_importance()`**:
     - Returns feature importance scores for treatment effect heterogeneity from `model_tau`.

### Workflow
1. **Propensity Score Estimation**: Predicts the probability of treatment assignment based on covariates.
2. **Survival Modeling**: Models survival probabilities considering both covariates and treatment.
3. **Treatment Effect Estimation**: Computes the difference in survival outcomes (RMST) between treated and control groups and models this effect across covariates.
4. **Prediction**: Provides treatment effect estimates or survival probabilities for new data.

### Use Case
This is useful in scenarios like medical trials, where you want to estimate how a treatment affects survival time across diverse patient profiles, accounting for censoring (e.g., patients lost to follow-up).

### Notes
- The implementation assumes binary treatment (`W = 0 or 1`).
- It uses honest splitting (`honesty=True`) to reduce bias in treatment effect estimates.
- The code is designed for integration with scikit-learn pipelines, supporting standard methods like `fit` and `predict`.

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sksurv.ensemble import RandomSurvivalForest
from sksurv.util import Surv
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.utils.validation import check_X_y, check_array
import warnings

class CausalSurvivalForest(BaseEstimator):
    """
    A Python implementation similar to R's grf causal_survival_forest.
    
    Parameters:
    -----------
    n_estimators : int (default=1000)
        Number of trees in the forest
    min_samples_leaf : int (default=10)
        Minimum number of samples in each leaf
    max_depth : int (default=None)
        Maximum depth of each tree
    random_state : int (default=None)
        Random seed for reproducibility
    honesty : bool (default=True)
        Whether to use honest splitting (subsample splitting)
    alpha : float (default=0.05)
        The fraction of samples used for honesty splitting
    verbose : bool (default=False)
        Whether to print progress during fitting
    """
    
    def __init__(self, n_estimators=1000, min_samples_leaf=10, max_depth=None,
                 random_state=None, honesty=True, alpha=0.05, verbose=False):
        self.n_estimators = n_estimators
        self.min_samples_leaf = min_samples_leaf
        self.max_depth = max_depth
        self.random_state = random_state
        self.honesty = honesty
        self.alpha = alpha
        self.verbose = verbose
        
        # Initialize models
        self.model_t = RandomForestRegressor(
            n_estimators=n_estimators,
            min_samples_leaf=min_samples_leaf,
            max_depth=max_depth,
            random_state=random_state
        )
        
        self.model_y = RandomSurvivalForest(
            n_estimators=n_estimators,
            min_samples_leaf=min_samples_leaf,
            max_depth=max_depth,
            random_state=random_state
        )
        
        self.model_tau = RandomForestRegressor(
            n_estimators=n_estimators,
            min_samples_leaf=min_samples_leaf,
            max_depth=max_depth,
            random_state=random_state
        )
        
        # Store unique event times after fitting
        self.unique_times_ = None
    
    def fit(self, X, W, Y, D, sample_weight=None):
        """
        Fit the causal survival forest.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Covariates/features
        W : array-like, shape (n_samples,)
            Treatment assignment (binary)
        Y : array-like, shape (n_samples,)
            Observed time
        D : array-like, shape (n_samples,)
            Event indicator (1 if event occurred, 0 if censored)
        sample_weight : array-like, shape (n_samples,), optional
            Sample weights
        """
        # Input validation
        X, W = check_X_y(X, W)
        Y, D = np.array(Y), np.array(D)
        
        if sample_weight is not None:
            sample_weight = np.array(sample_weight)
        
        # Store data
        self.X = X
        self.W = W
        self.Y = Y
        self.D = D
        self.sample_weight = sample_weight
        
        # Create survival structure for Y model
        y_struct = Surv.from_arrays(event=D, time=Y)
        
        # Step 1: Estimate propensity scores (probability of treatment)
        if self.verbose:
            print("Fitting treatment model...")
        self.model_t.fit(X, W)
        self.propensity_scores = self.model_t.predict(X)
        
        # Clip propensity scores to avoid extreme values
        self.propensity_scores = np.clip(self.propensity_scores, 0.1, 0.9)
        
        # Step 2: Estimate survival function
        if self.verbose:
            print("Fitting survival model...")
            
        # Add treatment as a feature for the survival model
        X_with_treatment = np.column_stack([X, W])
        self.model_y.fit(X_with_treatment, y_struct)
        
        # Store unique event times
        self.unique_times_ = np.unique(Y[D == 1])
        
        # Step 3: Estimate treatment effects (tau)
        if self.verbose:
            print("Fitting treatment effect model...")
            
        # Get survival predictions under treatment and control
        surv_control = self._predict_survival(X, W=0)
        surv_treated = self._predict_survival(X, W=1)
        
        # Calculate restricted mean survival time (RMST) difference as treatment effect
        rmst_control = np.trapz(surv_control, self.unique_times_)
        rmst_treated = np.trapz(surv_treated, self.unique_times_)
        tau = rmst_treated - rmst_control
        
        # Fit model to estimate heterogeneous treatment effects
        self.model_tau.fit(X, tau)
        
        return self
    
    def _predict_survival(self, X, W):
        """Predict survival probabilities for given treatment assignment"""
        # Create modified dataset with specified treatment
        X_mod = np.column_stack([X, np.full(X.shape[0], W)])
        
        # Get survival function predictions
        surv_funcs = self.model_y.predict_survival_function(X_mod)
        
        # Evaluate at unique event times
        surv_preds = np.zeros((X.shape[0], len(self.unique_times_)))
        for i, fn in enumerate(surv_funcs):
            surv_preds[i, :] = fn(self.unique_times_)
            
        return surv_preds
    
    def predict(self, X):
        """Predict treatment effects for new data"""
        X = check_array(X)
        return self.model_tau.predict(X)
    
    def predict_survival(self, X, W, times=None):
        """
        Predict survival probabilities for new data under specified treatment
        
        Parameters:
        -----------
        X : array-like
            Covariate matrix
        W : int or array-like
            Treatment assignment (0 or 1)
        times : array-like, optional
            Times at which to evaluate survival. If None, uses training event times.
        """
        X = check_array(X)
        
        if times is None:
            times = self.unique_times_
        
        # Create modified dataset with specified treatment
        if isinstance(W, (int, float)):
            W = np.full(X.shape[0], W)
        X_mod = np.column_stack([X, W])
        
        # Get survival function predictions
        surv_funcs = self.model_y.predict_survival_function(X_mod)
        
        # Evaluate at requested times
        surv_preds = np.zeros((X.shape[0], len(times)))
        for i, fn in enumerate(surv_funcs):
            surv_preds[i, :] = fn(times)
            
        return surv_preds
    
    def get_importance(self):
        """Get feature importance for treatment effect heterogeneity"""
        return self.model_tau.feature_importances_

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sksurv.ensemble import RandomSurvivalForest
from sksurv.util import Surv
from sklearn.base import BaseEstimator
from sklearn.utils.validation import check_X_y, check_array

# Load the lung dataset
url = "https://github.com/zia207/r-colab/raw/main/Data/Machine_Learning/lung.csv"
lung = pd.read_csv(url)

# Check for NAs in key variables
print("NAs in time:", lung['time'].isna().sum())
print("NAs in status:", lung['status'].isna().sum())
print(lung.head())

### Prepare the Data
covar_names = ["age", "ph.ecog", "ph.karno", "pat.karno", "meal.cal", "wt.loss"]

# Remove rows with missing values in any relevant variable
lung2 = lung.dropna(subset=["time", "status", "sex"] + covar_names)
print("Rows after removing NAs:", len(lung2))

# Prepare variables for CSF
Y = lung2['time'].values
D = np.where(lung2['status'] == 2, 1, 0)  # 1=event(death), 0=censored
W = np.where(lung2['sex'] == 2, 1, 0)    # 1=female, 0=male (treatment: female)
X = lung2[covar_names].values

# Check distribution of events and event times
print("Number of events (D=1):\n", pd.Series(D).value_counts())
print("Summary of event times where event observed:\n", pd.Series(Y[D == 1]).describe())

plt.hist(Y[D == 1], bins=20)
plt.title("Histogram of Event Times (Deaths)")
plt.xlabel("Survival Time (days)")
plt.show()

# Find a valid horizon (latest time with at least one event)
event_times = np.sort(np.unique(Y[D == 1]))
last_valid_ht = event_times[-1]
print("Using horizon =", last_valid_ht)

### Split Data into Training and Test Sets
X_train, X_test, Y_train, Y_test, W_train, W_test, D_train, D_test = train_test_split(
    X, Y, W, D, test_size=0.2, random_state=42
)

print("Training set size:", len(X_train))
print("Test set size:", len(X_test))
print("Column names of X:", covar_names)

### Train the Causal Survival Forest
# Initialize our custom CSF
csf_model = CausalSurvivalForest(
    n_estimators=1000,
    min_samples_leaf=5,
    max_depth=None,
    random_state=123,
    honesty=True,
    alpha=0.05,
    verbose=True
)

# Fit the model
csf_model.fit(X_train, W_train, Y_train, D_train)

# Predict treatment effects for training data
tau_hat_train = csf_model.predict(X_train)

# Calculate ATE on training data
ate_train = np.mean(tau_hat_train)
ate_train_se = np.std(tau_hat_train) / np.sqrt(len(tau_hat_train))
print(f"Training ATE: {ate_train:.2f} ± {1.96 * ate_train_se:.2f} (95% CI)")

### Predict Treatment Effects
# Predict treatment effects at the chosen horizon
# (Note: Our implementation predicts RMST difference by default)
tau_hat = csf_model.predict(X_test)

# Visualize estimated treatment effects
plt.hist(tau_hat, bins=20)
plt.title(f"Estimated Treatment Effects at {last_valid_ht:.0f} days")
plt.xlabel("Effect of being Female (vs Male) on RMST (days)")
plt.show()

print("Mean estimated treatment effect:", np.mean(tau_hat))

### Make Predictions
# Predict treatment effects for all patients
csf_pred = csf_model.predict(X)

# Plot histogram of the estimated treatment effects
plt.hist(csf_pred, bins=20)
plt.title("Distribution of Estimated Treatment Effects (RMST)")
plt.xlabel("Treatment Effect (Days)")
plt.show()

### Estimate the Average Treatment Effect
ate = np.mean(csf_pred)
ate_se = np.std(csf_pred) / np.sqrt(len(csf_pred))
print(f"ATE: {ate:.2f} ± {1.96 * ate_se:.2f} (95% CI)")

### Assess Heterogeneity (Simplified)
# For a proper evaluation, we'd need to implement RATE/AUTOC
# Here's a simplified version:

# Split data into training and evaluation sets
X_train2, X_eval, W_train2, W_eval, Y_train2, Y_eval, D_train2, D_eval = train_test_split(
    X, W, Y, D, test_size=0.5, random_state=123
)

# Train forests on training and evaluation sets
train_csf = CausalSurvivalForest(n_estimators=1000, random_state=123)
train_csf.fit(X_train2, W_train2, Y_train2, D_train2)

eval_csf = CausalSurvivalForest(n_estimators=1000, random_state=123)
eval_csf.fit(X_eval, W_eval, Y_eval, D_eval)

# Predict treatment effects from training model on evaluation set
tau_train_pred = train_csf.predict(X_eval)

# Calculate correlation between predictions
corr = np.corrcoef(tau_train_pred, eval_csf.predict(X_eval))[0, 1]
print(f"Treatment effect correlation between models: {corr:.2f}")

### Variable Importance
var_importance = csf_model.get_importance()
features = covar_names

# Create a DataFrame for plotting
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': var_importance
}).sort_values('Importance', ascending=True)

# Create horizontal bar plot
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.title("Variable Importance in Causal Survival Forest")
plt.xlabel("Importance")
plt.show()